# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902076


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:01<00:26,  1.14it/s]

Rendering models:   9%|▉         | 3/32 [00:05<00:49,  1.72s/it]

Rendering models:  16%|█▌        | 5/32 [00:05<00:33,  1.23s/it]

Rendering models:  19%|█▉        | 6/32 [00:07<00:35,  1.38s/it]

...

Rendering models:  78%|███████▊  | 25/32 [00:09<00:00,  7.76it/s]

Rendering models:  84%|████████▍ | 27/32 [00:09<00:00,  7.92it/s]

Rendering models:  91%|█████████ | 29/32 [00:09<00:00,  8.93it/s]

Rendering models:  97%|█████████▋| 31/32 [00:09<00:00,  9.64it/s]

equidad1                              0.000801
eatyourgreens                         0.000908
Thedunkmasta                          0.002598
OregonHiker                           0.001407
sum0                                  0.017227
not-logged-in-460b94aefeb0ead1b0f0    1.485858
47degreesnorth                        0.004448
ElisabethB                            0.155543
mlang2017                             0.009568
not-logged-in-783570dae2c2970c42b4    0.422290
smart16                               0.001512
smiralexx                             0.116809
not-logged-in-214814e52004b42491a4    0.126093
not-logged-in-9f30f411df9d28d2c541    0.002017
pangeli5                              0.000594
acapirala                             0.002942
Lavadude                              0.005032
helios1235                            0.000468
not-logged-in-8700ff8251e0704f745c    0.000534
Bambura                               0.002666
not-logged-in-36a6cd4ee9f5f5a4f610    0.045812
barrowwright 

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())